<a href="https://colab.research.google.com/github/alessandro-crescenzi/computer_vision_project/blob/main/Traffic_Sign_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install deeplake

In [ ]:
import deeplake
import torch
from torchvision import transforms, models

train = deeplake.load("hub://activeloop/gtsrb-train")
test = deeplake.load("hub://activeloop/gtsrb-test")


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

In [ ]:
tform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((48, 48)), 
    #transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])

train_loader = train.pytorch(num_workers=0, batch_size=4, transform={
                        'images': tform, 'labels': None}, shuffle=True)

test_loader = train.pytorch(num_workers=0, batch_size=4, transform={
                        'images': tform, 'labels': None}, shuffle=True)

In [6]:
net = models.resnet18()
net.fc = torch.nn.Linear(net.fc.in_features, len(train.labels.info.class_names))
net = net.to(device)
    
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        images, labels = data['images'].to(device), data['labels'].to(device)
        #print (labels)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(images)
        loss = criterion(outputs, labels.reshape(-1))
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

In [ ]:
import time

for i, data in enumerate(test_loader):
        images, labels = data['images'].to(device), data['labels'].to(device)

        outputs = net(images)

        pred = outputs.data.max(1, keepdim=True)[1] # get the index of the max log-probability

        #labels = labels[:, 0]
        #pred = pred[:, 0]

        correct = torch.eq(pred, labels)

        print ("Correct : " + str(correct))
        print ("Label:")
        print (labels)
        print ("Pred:")
        print (pred)
        time.sleep(10)


